# Análisis de la base de datos de una aplicación de libros
**Objetivo:** estudiar la base de datos mediante querys en SQL para generar una propuesta de valor.

**Tareas**
- Número de libros publicados después del 1 de enero de 2000.
- Número de reseñas de usuarios y la calificación promedio para cada libro.
- Identifica la editorial que ha publicado el mayor número de libros con más de 50 páginas (ayudará a excluir folletos y publicaciones similares del análisis).
- Identifica al autor que tiene la más alta calificación promedio del libro: mirar solo los libros con al menos 50 calificaciones.
- Encuentra el número promedio de reseñas de texto entre los usuarios que calificaron más de 50 libros.

## Diccionario de datos
**`books`**

Contiene datos sobre libros:

- `book_id`: identificación del libro
- `author_id`: identificación del autor o autora
- `title`: título
- `num_pages`: número de páginas
- `publication_date`: fecha de la publicación
- `publisher_id`: identificación de la editorial

**`authors`**

Contiene datos sobre autores:

- `author_id`: identificación del autor o autora
- `author`: el autor o la autora

**`publishers`**

Contiene datos sobre editoriales:

- `publisher_id`: identificación de la editorial
- `publisher`: la editorial

**`ratings`**

Contiene datos sobre las calificaciones de usuarios:

- `rating_id`: identificación de la calificación
- `book_id`: identificación del libro
- `username`: el nombre del usuario que revisó el libro
- `rating`: calificación

**`reviews`**

Contiene datos sobre las reseñas de los y las clientes:

- `review_id`: identificación de la reseña
- `book_id`: identificación del libro
- `username`: el nombre del usuario que revisó el libro
- `text`: el texto de la reseña

## Diagrama de datos
![alt text](db_diagram.png)

In [1]:
# importar librerías
from sqlalchemy import create_engine
from dotenv import load_dotenv
import os
import pandas as pd

In [2]:
# Leer credenciales
load_dotenv()

user = os.getenv("DB_USER")
password = os.getenv("DB_PASSWORD")
host = os.getenv("DB_HOST")
port = os.getenv("DB_PORT")
database = os.getenv("DB_NAME")


connection_string = 'postgresql://{}:{}@{}:{}/{}'.format(user,
                                                         password,
                                                         host,
                                                         port,
                                                         database)

engine = create_engine(connection_string, connect_args={'sslmode':'require'})

In [3]:
# función para crear el DataFrame de la tabla
def table_view(table):
    '''
    Esta función toma como parámetro el nombre de la tabla
    en la base de datos y la retorna como un DataFrame
    '''
    query = f'''
            SELECT
                *
            FROM 
                {table};
            '''

    df = pd.read_sql(query, engine)
    return df

In [4]:
# mostrar tablas
books = table_view('books')
authors = table_view('authors')
publishers = table_view('publishers')
ratings = table_view('ratings')
reviews = table_view('reviews')

print('Tabla de books')
print('\n',books.head())
print('\n','Tabla de authors')
print('\n',authors.head())
print('\n','Tabla de publishers')
print('\n',publishers.head())
print('\n','Tabla de ratings')
print('\n',ratings.head())
print('\n','Tabla de reviews')
print('\n',reviews.head())

Tabla de books

    book_id  author_id                                              title  \
0        1        546                                       'Salem's Lot   
1        2        465                 1 000 Places to See Before You Die   
2        3        407  13 Little Blue Envelopes (Little Blue Envelope...   
3        4         82  1491: New Revelations of the Americas Before C...   
4        5        125                                               1776   

   num_pages publication_date  publisher_id  
0        594       2005-11-01            93  
1        992       2003-05-22           336  
2        322       2010-12-21           135  
3        541       2006-10-10           309  
4        386       2006-07-04           268  

 Tabla de authors

    author_id                          author
0          1                      A.S. Byatt
1          2  Aesop/Laura Harris/Laura Gibbs
2          3                 Agatha Christie
3          4                   Alan Brennert
4   

In [5]:
# función para visualizar consultas
def query_view(query):
    '''
    Esta función toma como parámetro un string en SQL
    para realizar la query a la base de datos
    '''
    q = query
    df = pd.read_sql(q, engine)
    return df

### Tarea 1: Número de libros publicados después del 1 de enero de 2000.

In [6]:
books_since_2000 = query_view(
    '''
    SELECT
        COUNT(book_id) AS num_books
    FROM
        books
    WHERE
        publication_date >= '2001-01-01';    
    '''
)
books_since_2000

,num_books
0,783


### Comentario
Se tiene en el catálogo 783 libros publicados desde 2001.

### Tarea 2: Número de reseñas de usuarios y la calificación promedio para cada libro.

In [7]:
books_ratings_reviews = query_view(
  '''
  WITH avg_ratings AS (
    SELECT
      book_id,
      AVG(rating) AS avg_rating
    FROM
      ratings
    GROUP BY
      book_id
  ),
  num_reviews AS (
    SELECT
      book_id,
      COUNT(review_id) AS num_reviews
    FROM
      reviews
    GROUP BY
      book_id
    )

  SELECT
    books.book_id AS book_id,
    books.title AS title,
    avg_ratings.avg_rating AS avg_ratings,
    num_reviews.num_reviews AS num_reviews
  FROM
    books
    INNER JOIN avg_ratings ON avg_ratings.book_id = books.book_id
    INNER JOIN num_reviews ON num_reviews.book_id = books.book_id;
  '''
)
books_ratings_reviews

,book_id,title,avg_ratings,num_reviews
0,1,'Salem's Lot,3.666667,2
1,2,1 000 Places to See Before You Die,2.500000,1
2,3,13 Little Blue Envelopes (Little Blue Envelope...,4.666667,3
3,4,1491: New Revelations of the Americas Before C...,4.500000,2
4,5,1776,4.000000,4
...,...,...,...,...
989,996,Wyrd Sisters (Discworld #6; Witches #2),3.666667,3
990,997,Xenocide (Ender's Saga #3),3.400000,3
991,998,Year of Wonders,3.200000,4
992,999,You Suck (A Love Story #2),4.500000,2


### Comentario
* Dentro del top 10 libros mejor rankeados (5.0) hay en su mayoría 2 reseñas. 
* Dentro del top 10 libros con mayor número de reseñas tienen una calificación promedio de 4. 

### Tarea 3: Identificar la editorial que ha publicado el mayor número de libros con más de 50 páginas.

In [8]:
publications_per_publisher = query_view(
    '''
    SELECT
        publishers.publisher AS publisher,
        COUNT(books.book_id) AS num_publications
    FROM
        books
        LEFT JOIN publishers ON publishers.publisher_id = books.publisher_id
    WHERE
        books.num_pages >= 50    
    GROUP BY
        publisher
    ORDER BY
        num_publications DESC
    LIMIT 10;
    '''
)
publications_per_publisher

,publisher,num_publications
0,Penguin Books,42
1,Vintage,31
2,Grand Central Publishing,25
3,Penguin Classics,24
4,Ballantine Books,19
5,Bantam,19
6,Berkley,17
7,St. Martin's Press,14
8,Berkley Books,14
9,William Morrow Paperbacks,13


### Comentario
La editorial con mayor número de publicaciones es "Penguin Books"

### Tarea 4: Identificar al autor que tiene la más alta calificación promedio del libro: solo libros con al menos 50 calificaciones.

In [9]:
top_5_authors = query_view(
    '''
    WITH ratings_50 AS (
        SELECT
            book_id,
            AVG(rating) AS avg_rating,
            COUNT(rating_id) AS num_reviews
        FROM
            ratings
        GROUP BY
            book_id
        HAVING
            COUNT(rating_id) > 50
    )
    
    SELECT
        authors.author AS author,
        AVG(ratings_50.avg_rating) AS author_rating
    FROM
        books
        INNER JOIN ratings_50 ON ratings_50.book_id = books.book_id
        INNER JOIN authors ON authors.author_id = books.author_id
    GROUP BY
        author
    ORDER BY
        author_rating DESC
    LIMIT 5;
    '''
)
top_5_authors

,author,author_rating
0,J.K. Rowling/Mary GrandPré,4.283844
1,Markus Zusak/Cao Xuân Việt Khương,4.264151
2,J.R.R. Tolkien,4.258446
3,Louisa May Alcott,4.192308
4,Rick Riordan,4.080645


### Comentario
La autora con mayor calificación es J.K. Rowling, aunque tanto Markus Zusak como J.R.R. Tolkien tienen una tienen una califiación muy cercana a Rowling.

### Tarea 5: Encuentra el número promedio de reseñas de texto entre los usuarios que calificaron más de 50 libros.

In [10]:
query_view(
    '''    
    WITH users_50_ratings AS (
        SELECT
            username,
            COUNT(book_id) AS num_ratings
        FROM
            ratings
        GROUP BY
            username
        HAVING
            COUNT(book_id) >= 50
    ),
    users_reviews AS (
        SELECT
            username,
            COUNT(book_id) AS num_reviews
        FROM
            reviews
        GROUP BY
            username
    )
    
    SELECT
        AVG(users_reviews.num_reviews) AS avg_reviews_per_user
    FROM
        users_50_ratings
        LEFT JOIN users_reviews ON users_reviews.username = users_50_ratings.username;
    '''
)

,avg_reviews_per_user
0,24.222222


### Comentario
Los usuarios con mayor actividad en la califiación de libros realizan alrededor de 24 reseñas cada uno.

### Comentarios finales
Tras analizar los datos de reseñas y calificaciones promedio, propongo considerar simultáneamente el número de reseñas y la calificación promedio para definir el apartado de “Top 10 mejores libros”. Aquellos títulos con un número reducido de reseñas pero con altas calificaciones podrían incluirse en una categoría especial denominada “Libros por descubrir”, lo que permitiría destacarlos sin sesgar el ranking principal.

Para incrementar la participación de los usuarios, una buena estrategia podría ser mostrar una ventana emergente o enviar un correo electrónico cuando el lector haya concluido un libro o haya avanzado al menos el 80% de este, invitándole a dejar una calificación o reseña. Esto ayudaría a aumentar el volumen de retroalimentación disponible. Además, se podrían segmentar los perfiles de usuarios que generan más reseñas y calificaciones para recomendarles más títulos afines y fomentar su participación.

Finalmente, los autores mejor calificados pueden utilizarse como punto de partida para identificar patrones entre autores, géneros y editoriales. Esto facilitaría la detección de tendencias y contribuiría a mejorar el algoritmo de recomendaciones; por ejemplo, analizando qué autores o editoriales destacan dentro del género thriller.